In [1]:
%load_ext nb_black

import os
import sys

sys.path.append(
    os.path.abspath("/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/codebase")
)
import torch.backends.cudnn as cudnn
import random
from datetime import datetime
import time
from collections import OrderedDict
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import one_hot
import utils

from Explainer.loss_F import loss_fn_kd, entropy_loss
from Explainer.models.Gated_Logic_Net import Gated_Logic_Net
from Explainer.models.explainer import Explainer
from Explainer.models.pi import Pi
from dataset.dataset_cubs import Dataset_cub_for_explainer
from dataset.utils_dataset import get_dataset_with_image_and_attributes
from Explainer.loss_F import loss_fn_kd, entropy_loss, Selective_Distillation_Loss
from Explainer.models.concepts import Conceptizator
import jupyter_notebook.stats_t as stats
import matplotlib.pyplot as plt
import warnings
import sklearn.metrics as metrics

# from ipython_utils import cal_acc, show_results
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.stats import percentileofscore
from sklearn.metrics import roc_auc_score, accuracy_score

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [2]:
disease = "cardiomegaly"
path = f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/{disease}"
test_attr = torch.load(os.path.join(path, "dataset_g", "test_attributes.pt"))
test_proba_concept = torch.load(os.path.join(path, "dataset_g", "test_proba_concepts.pt"))
test_logits_concept = torch.load(os.path.join(path, "dataset_g", "test_logits_concepts.pt"))
test_logits_bb = torch.load(os.path.join(path, "dataset_g", "test_logits_bb.pt"))
test_class_labels = torch.load(os.path.join(path, "dataset_g", "test_class_labels.pt"))

train_attr = torch.load(os.path.join(path, "dataset_g", "train_attributes.pt"))
train_proba_concept = torch.load(os.path.join(path, "dataset_g", "train_proba_concepts.pt"))
train_logits_concept = torch.load(os.path.join(path, "dataset_g", "train_logits_concepts.pt"))
train_logits_bb = torch.load(os.path.join(path, "dataset_g", "train_logits_bb.pt"))
train_class_labels = torch.load(os.path.join(path, "dataset_g", "train_class_labels.pt"))

val_attr = torch.load(os.path.join(path, "dataset_g", "val_attributes.pt"))
val_proba_concept = torch.load(os.path.join(path, "dataset_g", "val_proba_concepts.pt"))
val_logits_concept = torch.load(os.path.join(path, "dataset_g", "val_logits_concepts.pt"))
val_logits_bb = torch.load(os.path.join(path, "dataset_g", "val_logits_bb.pt"))
val_class_labels = torch.load(os.path.join(path, "dataset_g", "val_class_labels.pt"))

print(
    test_attr.size(), test_proba_concept.size(), test_logits_concept.size(),
    test_logits_bb.size(), test_class_labels.size()
)
print()
print(
    train_attr.size(), train_proba_concept.size(), train_logits_concept.size(),
    train_logits_bb.size(), train_class_labels.size()
)
print()
print(
    val_attr.size(), val_proba_concept.size(), val_logits_concept.size(),
    val_logits_bb.size(), val_class_labels.size()
)


torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([48144]) torch.Size([24072, 1])

torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([375096]) torch.Size([187548, 1])

torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([46898]) torch.Size([23449, 1])


<IPython.core.display.Javascript object>

In [6]:
print(test_attr.device)
print(test_proba_concept.device)
print(test_logits_concept.device)
print(test_logits_bb.device)
print(test_class_labels.device)

cpu
cpu
cpu
cpu
cpu


<IPython.core.display.Javascript object>

In [7]:
with open(os.path.join(path, "MIMIC_test_configs.pkl"), "rb") as input_file:
    args = pickle.load(input_file)

concepts = args.landmark_names_spec + args.abnorm_obs_concepts

<IPython.core.display.Javascript object>

In [8]:
concepts.index(disease)

60

<IPython.core.display.Javascript object>

In [9]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
auroc = []
recall = []

for i in range(len(concepts)):
    try:
        auroc.append(metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i]))
        recall.append(metrics.recall_score(data_GT[:, i], y_pred[:, i]))
    except ValueError:
        pass

print(len(auroc))

mean_auroc = np.mean(np.array(auroc))
mean_recall = np.mean(np.array(recall))
mean_auroc, mean_recall

107


(0.719555520328648, 0.6094397876977458)

<IPython.core.display.Javascript object>

In [10]:
mean_auroc = np.mean(np.array(auroc))
mean_auroc

0.719555520328648

<IPython.core.display.Javascript object>

In [11]:
disease_index = concepts.index(disease)
concepts.index(disease), concepts[concepts.index(disease)]

(60, 'cardiomegaly')

<IPython.core.display.Javascript object>

In [12]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
sub_auroc = []
sub_recall = []
all_auroc = []
cnt = 0
for i in range(len(concepts)):
    auroc_val = metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i])
    recall_val = metrics.recall_score(data_GT[:, i], y_pred[:, i])
    all_auroc.append(auroc_val)
    if auroc_val >= 0.7 and i != disease_index:
        print(f"{concepts[i]}({i}), auroc: {auroc_val}, recall: {recall_val}")
        sub_auroc.append(auroc_val)
        sub_recall.append(recall_val)
        cnt += 1

print(cnt)
np.mean(np.array(sub_auroc)), np.mean(np.array(sub_recall))

hilar_unspec(3), auroc: 0.7111611979327944, recall: 0.6563517915309446
left_pleural(4), auroc: 0.8237385888786507, recall: 0.7612559241706162
right_pleural(5), auroc: 0.8347063341857299, recall: 0.7708565072302559
pleural_unspec(6), auroc: 0.8058717908112611, recall: 0.6637318943481966
heart_size(7), auroc: 0.7658428910303774, recall: 0.8036219581211093
left_diaphragm(9), auroc: 0.7843479425585844, recall: 0.6795180722891566
right_diaphragm(10), auroc: 0.8262234175745069, recall: 0.6107784431137725
retrocardiac(12), auroc: 0.7474675033234892, recall: 0.7307979120059657
left_apical_lung(21), auroc: 0.705325798034813, recall: 0.5612903225806452
left_lung_unspec(22), auroc: 0.7144111147484766, recall: 0.6671368124118476
right_apical_lung(26), auroc: 0.700273269395596, recall: 0.5773672055427251
right_lung_unspec(27), auroc: 0.7148470929694971, recall: 0.7534318901795143
lung_apices(28), auroc: 0.7050514652650215, recall: 0.5634920634920635
lung_bases(29), auroc: 0.7178861224704912, recall

(0.7726141688950959, 0.6932345519780105)

<IPython.core.display.Javascript object>

In [13]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
bad_auroc = []
bad_recall = []
cnt = 0
for i in range(len(concepts)):
    auroc_val = metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i])
    recall_val = metrics.recall_score(data_GT[:, i], y_pred[:, i])
    if auroc_val <= 0.7:
        print(f"{concepts[i]}({i}), auroc: {auroc_val}, recall: {recall_val}")
        cnt += 1
        bad_auroc.append(auroc_val)
        bad_recall.append(recall_val)

print(cnt)
np.mean(np.array(sub_auroc)), np.mean(np.array(sub_recall))

trachea(0), auroc: 0.6088573246824026, recall: 0.30196078431372547
left_hilar(1), auroc: 0.6928121140975408, recall: 0.5363636363636364
right_hilar(2), auroc: 0.6542108933277114, recall: 0.5627240143369175
heart_border(8), auroc: 0.6258498426978677, recall: 0.37305699481865284
diaphragm_unspec(11), auroc: 0.6985448908426588, recall: 0.7057387057387058
lower_left_lobe(13), auroc: 0.6956058678956796, recall: 0.7339821573398215
upper_left_lobe(14), auroc: 0.661523936361448, recall: 0.5846560846560847
lower_right_lobe(15), auroc: 0.6694290223998072, recall: 0.4491758241758242
middle_right_lobe(16), auroc: 0.6608848878016607, recall: 0.5705882352941176
upper_right_lobe(17), auroc: 0.6996044942811842, recall: 0.38445378151260506
left_lower_lung(18), auroc: 0.6583255414699264, recall: 0.5418060200668896
left_mid_lung(19), auroc: 0.5881714726127516, recall: 0.4619289340101523
left_upper_lung(20), auroc: 0.6652825337595705, recall: 0.37373737373737376
right_lower_lung(23), auroc: 0.680957028801

(0.7726141688950959, 0.6932345519780105)

<IPython.core.display.Javascript object>

In [16]:
mask_select = np.array(all_auroc) >= 0.7
mask_select[disease_index] = False

torch_mask = torch.from_numpy(mask_select)
torch_mask.sum(), torch_mask.size()

(tensor(60), torch.Size([107]))

<IPython.core.display.Javascript object>

In [17]:
cnt = 0
th = 0.7
concept_select = []
metric = "auroc"

test_select_attr = []
test_select_proba_concept = []
test_select_logits_concept = []


val_select_attr = []
val_select_proba_concept = []
val_select_logits_concept = []


train_select_attr = []
train_select_proba_concept = []
train_select_logits_concept = []


test_select_proba_concept = []
for i in range(len(concepts)):
    val = auroc[i] if metric == "auroc" else recall[i]

    if val >= th and concepts[i] != disease:
        test_select_attr.append(test_attr[:, i])
        test_select_proba_concept.append(test_proba_concept[:, i])
        test_select_logits_concept.append(test_logits_concept[:, i])

        val_select_attr.append(val_attr[:, i])
        val_select_proba_concept.append(val_proba_concept[:, i])
        val_select_logits_concept.append(val_logits_concept[:, i])

        train_select_attr.append(train_attr[:, i])
        train_select_proba_concept.append(train_proba_concept[:, i])
        train_select_logits_concept.append(train_logits_concept[:, i])

        concept_select.append(concepts[i])
        cnt += 1

test_select_attr = torch.stack(test_select_attr, dim=1)
test_select_proba_concept = torch.stack(test_select_proba_concept, dim=1)
test_select_logits_concept = torch.stack(test_select_logits_concept, dim=1)
val_select_attr = torch.stack(val_select_attr, dim=1)
val_select_proba_concept = torch.stack(val_select_proba_concept, dim=1)
val_select_logits_concept = torch.stack(val_select_logits_concept, dim=1)
train_select_attr = torch.stack(train_select_attr, dim=1)
train_select_proba_concept = torch.stack(train_select_proba_concept, dim=1)
train_select_logits_concept = torch.stack(train_select_logits_concept, dim=1)


print("Test sizes: ")
print(test_select_attr.size())
print(test_select_proba_concept.size())
print(test_select_logits_concept.size())
print("Val sizes: ")
print(val_select_attr.size())
print(val_select_proba_concept.size())
print(val_select_logits_concept.size())
print("Train sizes: ")
print(train_select_attr.size())
print(train_select_proba_concept.size())
print(train_select_logits_concept.size())
print(cnt)

Test sizes: 
torch.Size([24072, 60])
torch.Size([24072, 60])
torch.Size([24072, 60])
Val sizes: 
torch.Size([23449, 60])
torch.Size([23449, 60])
torch.Size([23449, 60])
Train sizes: 
torch.Size([187548, 60])
torch.Size([187548, 60])
torch.Size([187548, 60])
60


<IPython.core.display.Javascript object>

In [18]:
metric = "auroc"

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"torch_concepts_mask_{metric}.pt"),
    tensor_to_save=torch_mask,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_{metric}_attributes.pt"),
    tensor_to_save=test_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_proba_{metric}_concepts.pt"),
    tensor_to_save=test_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_logits_{metric}_concepts.pt"),
    tensor_to_save=test_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_{metric}_attributes.pt"),
    tensor_to_save=val_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_proba_{metric}_concepts.pt"),
    tensor_to_save=val_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_logits_{metric}_concepts.pt"),
    tensor_to_save=val_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_{metric}_attributes.pt"),
    tensor_to_save=train_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_proba_{metric}_concepts.pt"),
    tensor_to_save=train_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_logits_{metric}_concepts.pt"),
    tensor_to_save=train_select_logits_concept,
)

pickle.dump(
    concept_select,
    open(os.path.join(path, "dataset_g", f"selected_concepts_{metric}.pkl"), "wb"),
)

print(os.path.join(path, "dataset_g"))

/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/cardiomegaly/dataset_g


<IPython.core.display.Javascript object>

In [12]:
# concept_select.remove("cardiomegaly")
concept_select

['hilar_unspec',
 'left_pleural',
 'right_pleural',
 'pleural_unspec',
 'heart_size',
 'left_diaphragm',
 'right_diaphragm',
 'retrocardiac',
 'left_apical_lung',
 'left_lung_unspec',
 'right_apical_lung',
 'right_lung_unspec',
 'lung_apices',
 'lung_bases',
 'mediastinal',
 'spine',
 'right_atrium',
 'right_ventricle',
 'aorta',
 'svc',
 'interstitium',
 'parenchymal',
 'stomach',
 'effusion',
 'edema',
 'atelectasis',
 'tube',
 'consolidation',
 'process',
 'enlarge',
 'tip',
 'low',
 'line',
 'congestion',
 'catheter',
 'tortuous',
 'lead',
 'disease',
 'calcification',
 'device',
 'engorgement',
 'elevation',
 'wire',
 'degenerative',
 'pacemaker',
 'scar',
 'widen',
 'collapse',
 'emphysema',
 'mass',
 'crowd',
 'infiltrate',
 'obscure',
 'deformity',
 'hernia',
 'drainage',
 'distention',
 'shift',
 'pressure',
 'chf']

<IPython.core.display.Javascript object>

In [13]:
y_gt_tr = torch.load(os.path.join(path, "dataset_g", f"train_class_labels.pt"))
(y_gt_tr == 1).sum(dim=0), (y_gt_tr == 0).sum(dim=0)

(tensor([26196]), tensor([161352]))

<IPython.core.display.Javascript object>

In [14]:
((y_gt_tr == 1).sum(dim=0) / y_gt_tr.size(0)).item()

0.13967624306678772

<IPython.core.display.Javascript object>